In [ ]:
# Cell 1 – Mount Drive | Imports | Seed | CONFIG
from pathlib import Path
import os, time, math, pickle, random, warnings
import numpy as np
import torch, torch.nn as nn, torch.optim as optim, torch.nn.functional as F
from tqdm import tqdm  # Add tqdm for progress tracking
warnings.filterwarnings("ignore")

# Mount Google Drive
def in_colab():
    try:
        import google.colab  # noqa
        return True
    except Exception:
        return False

if in_colab():
    from google.colab import drive
    drive.mount("/content/drive", force_remount=False)

# Reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    if torch.cuda.is_available():
        torch.backends.cudnn.deterministic = False
        torch.backends.cudnn.benchmark = True

set_seed(42)

# CONFIG Declaration
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"PyTorch CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Current CUDA device: {torch.cuda.current_device()}")
    print(f"CUDA device count: {torch.cuda.device_count()}")

CONFIG = {
    "DEVICE": DEVICE,
    "SEED": 42,
    "NUM_WORKERS": 0,
    "BATCH_SIZE": 32 if DEVICE == "cpu" else 64,
    "LR": 1e-3,
    "DEFAULT_EPOCHS": 30,
    "EPOCHS_PER_DATASET": {"CIFAR10": 15, "CIFAR100": 30, "CINIC10": 30, "YAHOO":10,"CRITEO":15},
    "KD_TEMPERATURE": 4.0,
    "KD_LAMBDA": 0.7,
    "AT_PGD_STEPS": 5,
    "AT_STEP_SIZE": 2/255,
    "AUX_SIZE": 1000,
    "DATASET_HP": {
        "CIFAR10":  {"tau": 4.0, "k": 3, "eps_k": 0.15, "lr_teacher": 1e-3, "lr_student": 5e-4, "adv_eps": 8/255},
        "CIFAR100": {"tau": 5.0, "k": 5, "eps_k": 0.15, "lr_teacher": 1e-3, "lr_student": 5e-4, "adv_eps": 8/255},
        "CINIC10":  {"tau": 3.5, "k": 3, "eps_k": 0.15, "lr_teacher": 1e-3, "lr_student": 5e-4, "adv_eps": 8/255},
         "YAHOO":    {"tau": 4.0, "k": 5, "eps_k": 0.1, "lr_teacher": 5e-4, "lr_student": 1e-4, "adv_eps": 0.05},
        "CRITEO":   {"tau": 4.0, "k": 5, "eps_k": 0.1, "lr_teacher": 1e-3, "lr_student": 5e-4, "adv_eps": 0.1}
    },

    "SIZES": {
        "CIFAR10":  (50_000, 10_000),
        "CIFAR100": (50_000, 10_000),
        "CINIC10":  (90_000, 90_000),
        "YAHOO":    (50_000, 20_000),
        "CRITEO":   (80_000, 20_000)
    },
    "CLAMP_MIN": -3.0,
    "CLAMP_MAX":  3.0,
    "DRIVE_PATH": "/content",
    "PIN_MEMORY": False,
    "PREFETCH_FACTOR": None,

}

os.makedirs(CONFIG["DRIVE_PATH"], exist_ok=True)
print("Device:", CONFIG["DEVICE"])
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory/1e9:.2f} GB")
    torch.backends.cudnn.benchmark = True
    print("cuDNN benchmark enabled for GPU performance.")
else:
    print("WARNING: Running on CPU - training will be slow!")

Mounted at /content/drive
PyTorch CUDA available: True
Current CUDA device: 0
CUDA device count: 1
Device: cuda
GPU: Tesla T4
GPU Memory: 15.83 GB
cuDNN benchmark enabled for GPU performance.


In [ ]:
# Cell 2 - VFL Training, LIA/Adversarial Attack and Evaluation Functions

from tqdm import tqdm
import torch
import torch.nn.functional as F
from sklearn.metrics import f1_score, roc_auc_score, top_k_accuracy_score
import numpy as np

# Generate_adv_examples function remains the same
def generate_adv_examples(model, xa, xp, y, config, dataset_name):
    hp = config["DATASET_HP"][dataset_name]
    eps = hp["adv_eps"]
    is_vision = dataset_name in ["CIFAR10", "CIFAR100", "CINIC10"]
    xp_adv = xp.clone().detach().requires_grad_(True)
    original_mode = model.training
    model.eval()
    with torch.enable_grad():
        logits = model(xa, xp_adv)
        loss = F.cross_entropy(logits, y)
    grad = torch.autograd.grad(loss, xp_adv, retain_graph=False, create_graph=False)[0]
    model.train(original_mode)
    if is_vision:
        perturbation = eps * grad.sign()
    else:
        grad_norm = torch.norm(grad.view(grad.shape[0], -1), p=2, dim=1).view(-1, 1, 1, 1) if grad.dim() == 4 else torch.norm(grad, p=2, dim=1).view(-1, 1)
        perturbation = eps * grad / (grad_norm + 1e-8)
    xp_perturbed = xp + perturbation
    if is_vision:
        xp_perturbed = torch.clamp(xp_perturbed, config["CLAMP_MIN"], config["CLAMP_MAX"])
    return xp_perturbed.detach()


def train_epoch_unified(model, loader, optimizer, config, dataset_name, teacher=None, use_at=False, use_dp=False, noise_multiplier=0.3, use_embedding_noise=False, embedding_noise_level=0.1, sched=None):
    """A unified training function with correct metric reporting."""
    model.train()
    if teacher: teacher.eval()

    total_loss, total_correct, total_samples = 0, 0, 0 # Initialize metrics
    iterator = tqdm(loader, desc=f'Training {dataset_name}', ncols=110)
    device = config["DEVICE"]
    hp = config["DATASET_HP"][dataset_name]

    for (xa, xp), y in iterator:
        xa, xp, y = xa.to(device), xp.to(device), y.to(device)
        optimizer.zero_grad()

        xp_train = generate_adv_examples(model, xa, xp, y, config, dataset_name) if use_at else xp

        h_a = model.bottom_a(xa)
        h_p = model.bottom_p(xp_train)

        if use_embedding_noise:
            h_p = h_p + torch.randn_like(h_p) * embedding_noise_level

        if use_dp:
            def add_noise_hook(grad):
                noise = torch.randn_like(grad) * noise_multiplier
                return grad + noise
            h_p.register_hook(add_noise_hook)

        logits = model.top_model(h_a, h_p)

        if teacher:
            with torch.no_grad():
                teacher_logits = teacher(xa, xp_train)
            T = hp["tau"]
            kd_loss = F.kl_div(F.log_softmax(logits / T, dim=1), F.log_softmax(teacher_logits / T, dim=1), reduction='batchmean', log_target=True) * (T * T)
            ce_loss = F.cross_entropy(logits, y)
            loss = config["KD_LAMBDA"] * kd_loss + (1 - config["KD_LAMBDA"]) * ce_loss
        else:
            loss = F.cross_entropy(logits, y)

        loss.backward()
        optimizer.step()
        if sched: sched.step()

        total_loss += loss.item() * y.size(0)
        total_correct += (logits.argmax(1) == y).sum().item()
        total_samples += y.size(0)
        iterator.set_postfix({'loss': f'{loss.item():.4f}'})

    # Returns calculated average loss and accuracy
    avg_loss = total_loss / total_samples if total_samples > 0 else 0.0
    train_acc = 100 * total_correct / total_samples if total_samples > 0 else 0.0
    return avg_loss, train_acc

def evaluate_model_unified(model, loader, config, dataset_name, robust=False):
    model.eval()
    y_true_list, y_probs_list = [], []
    total_loss, total = 0, 0
    device = config["DEVICE"]
    with torch.no_grad():
        for (xa, xp), y in loader:
            xa, xp, y = xa.to(device), xp.to(device), y.to(device)
            xp_eval = generate_adv_examples(model, xa, xp, y, config, dataset_name) if robust else xp
            logits = model(xa, xp_eval)
            loss = F.cross_entropy(logits, y)
            total_loss += loss.item() * y.size(0)
            total += y.size(0)
            y_true_list.extend(y.cpu().numpy())
            y_probs_list.extend(F.softmax(logits, dim=1).cpu().numpy())
    loss_avg = total_loss / total if total > 0 else 0.0
    y_true, y_probs = np.array(y_true_list), np.array(y_probs_list)
    num_classes = y_probs.shape[1]
    if num_classes == 2:
        preds = np.argmax(y_probs, axis=1)
        top1_acc = (preds == y_true).mean() * 100
        top5_acc = top1_acc
    else:
        top1_acc = top_k_accuracy_score(y_true, y_probs, k=1, labels=range(num_classes)) * 100
        top5_acc = top_k_accuracy_score(y_true, y_probs, k=min(5, num_classes), labels=range(num_classes)) * 100
    return loss_avg, top1_acc, top5_acc

def run_privacy_attack_unified(*args, **kwargs):
    return run_privacy_attack_vision_multimode(*args, **kwargs)

def run_privacy_attack_vision_multimode(vfl_model, train_loader, test_loader, config, num_classes, attack_type="passive", aux_batches_limit=4, attacker_lr=5e-4, attacker_epochs=2, active_eps=0.02, active_queries=2, perturbed_sigma=0.1):
    device = config["DEVICE"]
    vfl_model.eval()
    (xa_ex, xp_ex), _ = next(iter(test_loader))
    with torch.no_grad():
        h_p_dim = vfl_model.bottom_p(xp_ex[:1].to(device)).shape[1]
    feat_dim = h_p_dim * active_queries if attack_type == "active" else h_p_dim
    attacker = nn.Sequential(nn.Linear(feat_dim, 128), nn.ReLU(inplace=True), nn.Dropout(0.2), nn.Linear(128, num_classes)).to(device)
    opt = torch.optim.AdamW(attacker.parameters(), lr=attacker_lr, weight_decay=1e-4)
    aux_data = []
    for i, batch in enumerate(train_loader):
        if i >= aux_batches_limit: break
        aux_data.append(batch)
    attacker.train()
    for _ in range(attacker_epochs):
        for (xa, xp), y in aux_data:
            xa, xp, y = xa.to(device), xp.to(device), y.to(device)
            with torch.enable_grad():
                if attack_type == "direct":
                    feat = vfl_model.bottom_p(xp)
                elif attack_type == "active":
                    grads = []
                    h_a = vfl_model.bottom_a(xa)
                    for _ in range(active_queries):
                        delta = (torch.rand_like(xp) * 2 - 1) * active_eps
                        xp_q = torch.clamp(xp + delta, config["CLAMP_MIN"], config["CLAMP_MAX"])
                        h_p_q = vfl_model.bottom_p(xp_q); h_p_q.requires_grad_()
                        logits_q = vfl_model.top_model(h_a, h_p_q)
                        grad_q = torch.autograd.grad(logits_q.sum(), h_p_q, retain_graph=False)[0]
                        grads.append(grad_q)
                    feat = torch.cat(grads, dim=1)
                else:
                    h_p = vfl_model.bottom_p(xp); h_p.requires_grad_()
                    h_a = vfl_model.bottom_a(xa)
                    logits = vfl_model.top_model(h_a, h_p)
                    grad = torch.autograd.grad(logits.sum(), h_p, retain_graph=False)[0]
                    if attack_type == "passive": feat = grad
                    elif attack_type == "perturbed": feat = grad + torch.randn_like(grad) * perturbed_sigma
            opt.zero_grad(set_to_none=True)
            pred = attacker(feat.detach())
            loss = F.cross_entropy(pred, y)
            loss.backward()
            opt.step()
    attacker.eval()
    y_true, y_probs = [], []
    with torch.no_grad():
        for (xa, xp), y in test_loader:
            xa, xp, y = xa.to(device), xp.to(device), y.to(device)
            with torch.enable_grad():
                if attack_type == "direct": feat = vfl_model.bottom_p(xp)
                elif attack_type == "active":
                    grads, h_a = [], vfl_model.bottom_a(xa)
                    for _ in range(active_queries):
                        delta = (torch.rand_like(xp) * 2 - 1) * active_eps
                        xp_q = torch.clamp(xp + delta, config["CLAMP_MIN"], config["CLAMP_MAX"])
                        h_p_q = vfl_model.bottom_p(xp_q); h_p_q.requires_grad_()
                        logits_q = vfl_model.top_model(h_a, h_p_q)
                        grad_q = torch.autograd.grad(logits_q.sum(), h_p_q, retain_graph=False)[0]
                        grads.append(grad_q)
                    feat = torch.cat(grads, dim=1)
                else:
                    h_p = vfl_model.bottom_p(xp); h_p.requires_grad_()
                    h_a = vfl_model.bottom_a(xa)
                    logits = vfl_model.top_model(h_a, h_p)
                    grad = torch.autograd.grad(logits.sum(), h_p, retain_graph=False)[0]
                    if attack_type == "passive": feat = grad
                    elif attack_type == "perturbed": feat = grad
            probs = F.softmax(attacker(feat.detach()), dim=1).cpu()
            y_true.extend(y.cpu().tolist()); y_probs.extend(probs.tolist())
    y_true, y_probs = np.array(y_true), np.array(y_probs)
    if num_classes == 2:
        preds = np.argmax(y_probs, axis=1)
        asr_top1 = (preds == y_true).mean() * 100
        asr_top5 = asr_top1
    else:
        asr_top1 = top_k_accuracy_score(y_true, y_probs, k=1, labels=range(num_classes)) * 100
        asr_top5 = top_k_accuracy_score(y_true, y_probs, k=min(5, num_classes), labels=range(num_classes)) * 100
    return float(asr_top1), float(asr_top5)

print("Unified training, evaluation, and attack functions and metric reporting functions declared")

✓ Unified training, evaluation, and attack functions are ready (with correct metric reporting).


In [ ]:
# Cell 3 – Checkpoint Saving Utility
from pathlib import Path
import torch

def save_checkpoint(model, dataset_name, model_type, epoch, config):
    """
    Saves a model checkpoint to a dedicated checkpoints folder in Google Drive.

    Args:
        model (nn.Module): The model to be saved.
        dataset_name (str): Name of the dataset (e.g., "CIFAR10").
        model_type (str): Type of the model (e.g., "OA", "KDk").
        epoch (int): The current epoch number (0-indexed).
        config (dict): The main configuration dictionary.
    """
    try:
        # Define the base results directory from your config
        # This needs to handle both VISION and TABULAR_TEXT paths
        is_vision = dataset_name in ["CIFAR10", "CIFAR100", "CINIC10"]
        folder_name = "VISION" if is_vision else "TABULAR_TEXT"

        base_dir = Path(config["DRIVE_PATH"]) / "VFL_Results" / folder_name

        # Create a dedicated subdirectory for checkpoints
        checkpoint_dir = base_dir / "checkpoints"
        checkpoint_dir.mkdir(parents=True, exist_ok=True)

        # Construct a descriptive filename
        checkpoint_filename = f"{dataset_name}_{model_type}_epoch_{epoch+1}.pth"
        checkpoint_path = checkpoint_dir / checkpoint_filename

        # Save the model's state dictionary
        torch.save(model.state_dict(), checkpoint_path)

        print(f"    → Checkpoint saved: {checkpoint_path.name}")

    except Exception as e:
        print(f"    ⚠️ Error saving checkpoint: {e}")
        print("    Please ensure your Google Drive is mounted and accessible.")

print("Checkpoint saving utility is ready")

✓ Cell 6a: Checkpoint saving utility is ready.


In [ ]:
# Cell 4 – Text & Tabular Dataset Loading and Preprocessing
!pip install -q datasets sentence_transformers fastparquet

import pandas as pd
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import zipfile
import shutil
from pathlib import Path

# VFL Dataset Class for Text/Tabular
class VFLTextTabularDataset(Dataset):
    def __init__(self, Xa, Xp, y):
        self.Xa = torch.tensor(Xa, dtype=torch.float32)
        self.Xp = torch.tensor(Xp, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return (self.Xa[idx], self.Xp[idx]), self.y[idx]

# Main Loader Dictionary
TABULAR_TEXT_LOADERS = {}
DATA_PATH = Path("/content/datasets")
DATA_PATH.mkdir(parents=True, exist_ok=True)

# Yahoo! Answers (Text)
print("Preparing Yahoo! Answers dataset...")
try:
    dataset = load_dataset("yahoo_answers_topics")
    train_df = dataset['train'].to_pandas().sample(n=CONFIG["SIZES"]["YAHOO"][0], random_state=CONFIG["SEED"])
    test_df = dataset['test'].to_pandas().sample(n=CONFIG["SIZES"]["YAHOO"][1], random_state=CONFIG["SEED"])
    print("  Generating text embeddings with SentenceTransformer...")
    encoder = SentenceTransformer('all-mpnet-base-v2', device=CONFIG["DEVICE"])
    X_train_title = encoder.encode(train_df['question_title'].tolist(), show_progress_bar=True)
    X_train_content = encoder.encode(train_df['question_content'].tolist(), show_progress_bar=True)
    X_test_title = encoder.encode(test_df['question_title'].tolist(), show_progress_bar=True)
    X_test_content = encoder.encode(test_df['question_content'].tolist(), show_progress_bar=True)
    y_train, y_test = train_df['topic'].values, test_df['topic'].values
    train_ds, test_ds = VFLTextTabularDataset(X_train_title, X_train_content, y_train), VFLTextTabularDataset(X_test_title, X_test_content, y_test)
    TABULAR_TEXT_LOADERS["YAHOO"] = {
        "train": DataLoader(train_ds, batch_size=CONFIG["BATCH_SIZE"], shuffle=True, drop_last=True),
        "test": DataLoader(test_ds, batch_size=CONFIG["BATCH_SIZE"], shuffle=False),
        "num_classes": 10
    }
    print(f"  Yahoo! Answers loaded. Train={len(train_ds)}, Test={len(test_ds)}")
except Exception as e:
    print(f"  Could not load Yahoo! Answers dataset. Error: {e}")

# Criteo CTR (Tabular)
print("\nPreparing Criteo CTR dataset...")
criteo_zip = DATA_PATH / "criteo.zip"
criteo_path = DATA_PATH / "criteo"
drive_criteo_zip = Path("/content/drive/MyDrive/datasets/criteo.zip")

if drive_criteo_zip.exists() and not criteo_zip.exists():
    print("  Copying criteo.zip to local disk for speed...")
    shutil.copy(drive_criteo_zip, criteo_zip)

if criteo_zip.exists():
    if not criteo_path.exists():
        print(f"  Extracting Criteo from {criteo_zip}...")
        with zipfile.ZipFile(criteo_zip, 'r') as zf:
            zf.extractall(criteo_path)

    potential_nested_path = criteo_path / 'criteo'
    if potential_nested_path.exists() and potential_nested_path.is_dir():
        print("  Nested 'criteo' directory found. Adjusting path.")
        criteo_path = potential_nested_path

    try:
        train_df = pd.read_parquet(criteo_path / 'train.parquet').sample(n=CONFIG["SIZES"]["CRITEO"][0], random_state=CONFIG["SEED"])
        test_df = pd.read_parquet(criteo_path / 'test.parquet').sample(n=CONFIG["SIZES"]["CRITEO"][1], random_state=CONFIG["SEED"])

        dense_feature_cols = train_df.columns[1:14]

        scaler = StandardScaler()
        X_train_dense = scaler.fit_transform(train_df[dense_feature_cols].fillna(0))
        X_test_dense = scaler.transform(test_df[dense_feature_cols].fillna(0))

        y_train, y_test = train_df['label'].values, test_df['label'].values

        Xa_train, Xp_train = X_train_dense[:, :6], X_train_dense[:, 6:]
        Xa_test, Xp_test = X_test_dense[:, :6], X_test_dense[:, 6:]

        train_ds, test_ds = VFLTextTabularDataset(Xa_train, Xp_train, y_train), VFLTextTabularDataset(Xa_test, Xp_test, y_test)

        TABULAR_TEXT_LOADERS["CRITEO"] = {
            "train": DataLoader(train_ds, batch_size=CONFIG["BATCH_SIZE"], shuffle=True, drop_last=True),
            "test": DataLoader(test_ds, batch_size=CONFIG["BATCH_SIZE"], shuffle=False),
            "num_classes": 2
        }
        print(f"  Criteo CTR loaded. Train={len(train_ds)}, Test={len(test_ds)}")

    except Exception as e:
        print(f"  Could not process Criteo dataset. Error: {e}")
else:
    print("  Criteo dataset zip not found. Skipped.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 72.0 MB/s eta 0:00:00
Preparing Yahoo! Answers dataset...


README.md: 0.00B [00:00, ?B/s]

yahoo_answers_topics/train-00000-of-0000(…):   0%|          | 0.00/241M [00:00<?, ?B/s]

yahoo_answers_topics/train-00001-of-0000(…):   0%|          | 0.00/270M [00:00<?, ?B/s]

yahoo_answers_topics/test-00000-of-00001(…):   0%|          | 0.00/21.9M [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1563 [00:00<?, ?it/s]

Batches:   0%|          | 0/1563 [00:00<?, ?it/s]

Batches:   0%|          | 0/625 [00:00<?, ?it/s]

Batches:   0%|          | 0/625 [00:00<?, ?it/s]

  Yahoo! Answers loaded. Train=50000, Test=20000

Preparing Criteo CTR dataset...
  Copying criteo.zip to local disk for speed...
  Extracting Criteo from /content/datasets/criteo.zip...
  Nested 'criteo' directory found. Adjusting path.
  Criteo CTR loaded. Train=80000, Test=20000


In [ ]:
# Cell 5 – VFL Text/Tabular Top and Bottom Model Definitions
class BottomModel_TabularText(nn.Module):
    def __init__(self, in_features, layers=[512, 256]):
        super().__init__()
        net = []
        if layers:
            for i, out_features in enumerate(layers):
                net.append(nn.Linear(in_features, out_features))
                net.append(nn.BatchNorm1d(out_features))
                net.append(nn.ReLU(inplace=True))
                net.append(nn.Dropout(0.3))
                in_features = out_features
        else:
            net.append(nn.Linear(in_features, in_features))
            net.append(nn.ReLU(inplace=True))

        self.net = nn.Sequential(*net)

    def forward(self, x):
        return self.net(x)

class TopModel_TabularText(nn.Module):
    def __init__(self, in_features, num_classes, layers=[512, 256]):
        super().__init__()
        net = []
        for i, out_features in enumerate(layers):
            net.append(nn.Linear(in_features, out_features))
            net.append(nn.BatchNorm1d(out_features))
            net.append(nn.ReLU(inplace=True))
            in_features = out_features
        self.net = nn.Sequential(*net)
        self.classifier = nn.Linear(in_features, num_classes)

    def forward(self, h_a, h_p):
        h = torch.cat([h_a, h_p], dim=1)
        features = self.net(h)
        return self.classifier(features)

class VFLModel_TT(nn.Module):
    def __init__(self, bottom_a, bottom_p, top_model):
        super().__init__()
        self.bottom_a = bottom_a
        self.bottom_p = bottom_p
        self.top_model = top_model

    def forward(self, x_a, x_p):
        return self.top_model(self.bottom_a(x_a), self.bottom_p(x_p))

def build_vfl_model_tabular_text(dataset_name, num_classes):
    if dataset_name == "YAHOO":
        bottom_a = BottomModel_TabularText(in_features=768, layers=[256]) # Single hidden layer
        bottom_p = BottomModel_TabularText(in_features=768, layers=[256]) # Single hidden layer
        top_model = TopModel_TabularText(in_features=512, num_classes=num_classes, layers=[256, 128])

    elif dataset_name == "CRITEO":
        bottom_a = BottomModel_TabularText(in_features=6, layers=[]) # No hidden layers
        bottom_p = BottomModel_TabularText(in_features=7, layers=[]) # No hidden layers
        top_model = TopModel_TabularText(in_features=13, num_classes=num_classes, layers=[32, 16])

    else:
        raise ValueError(f"Unknown dataset name for model building: {dataset_name}")

    model = VFLModel_TT(bottom_a, bottom_p, top_model)
    return model.to(CONFIG["DEVICE"])

print("Text/Tabular model architectures defined")

✓ Cell 10: Optimized Text/Tabular model architectures are ready.


In [ ]:
# Cell 6 – Main Text/Tabular Training Loop
import warnings
import time
from pathlib import Path
import torch
import torch.optim as optim

warnings.filterwarnings("ignore", category=UserWarning)

TABTEXT_GROUP = ["YAHOO", "CRITEO"]
SAVED_MODELS_TABTEXT = {}

drive_results_dir_tt = Path(CONFIG["DRIVE_PATH"]) / "VFL_Results" / "TABULAR_TEXT"
drive_results_dir_tt.mkdir(parents=True, exist_ok=True)
print("Starting Text/Tabular VFL Training with Final Optimizations\n" + "="*70)

for dataset_name in TABTEXT_GROUP:
    # Setup
    loaders = TABULAR_TEXT_LOADERS[dataset_name]
    tr_loader, te_loader = loaders["train"], loaders["test"]
    num_classes = loaders["num_classes"]
    epochs = CONFIG["EPOCHS_PER_DATASET"].get(dataset_name, CONFIG["DEFAULT_EPOCHS"])
    hp = CONFIG["DATASET_HP"][dataset_name]
    SAVED_MODELS_TABTEXT[dataset_name] = {}
    patience = 3
    builder_fn = build_vfl_model_tabular_text

    # Phase 1: Original Architecture OA/Teacher
    print(f"\n{dataset_name} – Phase 1: Training OA (Teacher)")
    OA = builder_fn(dataset_name, num_classes)
    opt = optim.AdamW(OA.parameters(), lr=hp["lr_teacher"])
    best_val, no_imp = 0, 0
    for ep in range(epochs):
        t0 = time.perf_counter()
        train_loss, train_acc = train_epoch_unified(OA, tr_loader, opt, CONFIG, dataset_name)
        val_loss, val_acc, _ = evaluate_model_unified(OA, te_loader, CONFIG, dataset_name)
        elapsed = time.perf_counter() - t0
        print(f"  Epoch {ep+1:02d}/{epochs} | Train Loss={train_loss:.4f} | Train ACC={train_acc:6.2f}% | Val Loss={val_loss:.4f} | Val ACC={val_acc:6.2f}% | Time={elapsed:.1f}s")

        if val_acc > best_val:
            best_val, no_imp = val_acc, 0
            torch.save(OA.state_dict(), drive_results_dir_tt / f"{dataset_name}_OA_best.pth")
            print(f"    → Best model saved (Val ACC: {val_acc:.2f}%)")
        elif ep > 1:
            no_imp += 1
        if no_imp >= patience:
            print("  Early stopping triggered.")
            break
    OA.load_state_dict(torch.load(drive_results_dir_tt / f"{dataset_name}_OA_best.pth", map_location=CONFIG["DEVICE"]))
    SAVED_MODELS_TABTEXT[dataset_name]["OA"] = f"{dataset_name}_OA_best.pth"

    # Phase 2: Knowledge Distillation (KD) with k-anonymity
    print(f"\n{dataset_name} – Phase 2: Training KDk")
    KDk = builder_fn(dataset_name, num_classes)
    opt = optim.AdamW(KDk.parameters(), lr=hp["lr_student"])
    best_val, no_imp = 0, 0
    for ep in range(epochs):
        t0 = time.perf_counter()
        train_loss, train_acc = train_epoch_unified(KDk, tr_loader, opt, CONFIG, dataset_name, teacher=OA)
        val_loss, val_acc, _ = evaluate_model_unified(KDk, te_loader, CONFIG, dataset_name)
        elapsed = time.perf_counter() - t0
        print(f"  Epoch {ep+1:02d}/{epochs} | Train Loss={train_loss:.4f} | Train ACC={train_acc:6.2f}% | Val Loss={val_loss:.4f} | Val ACC={val_acc:6.2f}% | Time={elapsed:.1f}s")

        if val_acc > best_val:
            best_val, no_imp = val_acc, 0
            torch.save(KDk.state_dict(), drive_results_dir_tt / f"{dataset_name}_KDk_best.pth")
            print(f"    → Best model saved (Val ACC: {val_acc:.2f}%)")
        elif ep > 1:
            no_imp += 1
        if no_imp >= patience:
            print("  Early stopping triggered.")
            break
    KDk.load_state_dict(torch.load(drive_results_dir_tt / f"{dataset_name}_KDk_best.pth", map_location=CONFIG["DEVICE"]))
    SAVED_MODELS_TABTEXT[dataset_name]["KDk"] = f"{dataset_name}_KDk_best.pth"

    # Phase 3: KDk with Adversarial Training (KDk+AT)
    print(f"\n{dataset_name} – Phase 3: Training KDk+AT")
    KDkAT = builder_fn(dataset_name, num_classes)
    opt = optim.AdamW(KDkAT.parameters(), lr=hp["lr_student"])
    best_rob, no_imp = 0, 0
    for ep in range(epochs):
        t0 = time.perf_counter()
        train_loss, train_acc = train_epoch_unified(KDkAT, tr_loader, opt, CONFIG, dataset_name, teacher=OA, use_at=True)
        val_loss, val_acc, _ = evaluate_model_unified(KDkAT, te_loader, CONFIG, dataset_name)
        _, rob_acc, _ = evaluate_model_unified(KDkAT, te_loader, CONFIG, dataset_name, robust=True)
        elapsed = time.perf_counter() - t0
        print(f"  Epoch {ep+1:02d}/{epochs} | Train Loss={train_loss:.4f} | Train ACC={train_acc:6.2f}% | Val ACC={val_acc:6.2f}% | Robust={rob_acc:6.2f}% | Time={elapsed:.1f}s")

        if rob_acc > best_rob:
            best_rob, no_imp = rob_acc, 0
            torch.save(KDkAT.state_dict(), drive_results_dir_tt / f"{dataset_name}_KDkAT_best.pth")
            print(f"    → Best model saved (Robust ACC: {rob_acc:.2f}%)")
        elif ep > 1:
            no_imp += 1
        if no_imp >= patience:
            print("  Early stopping triggered.")
            break
    KDkAT.load_state_dict(torch.load(drive_results_dir_tt / f"{dataset_name}_KDkAT_best.pth", map_location=CONFIG["DEVICE"]))
    SAVED_MODELS_TABTEXT[dataset_name]["KDk+AT"] = f"{dataset_name}_KDkAT_best.pth"

    # Phase 4: KDk+AT+DP (KDk with Adversarial Training and Differential Privacy)
    print(f"\n{dataset_name} – Phase 4: Training KDk+AT+DP")
    KDkATDP = builder_fn(dataset_name, num_classes)
    KDkATDP.load_state_dict(torch.load(drive_results_dir_tt / f"{dataset_name}_KDkAT_best.pth", map_location=CONFIG["DEVICE"]))

    if dataset_name == "YAHOO":
        finetune_lr = hp["lr_student"] / 25
        grad_noise = 4.0
        embed_noise = 0.3
        finetune_epochs = 20
        use_embed_noise_flag = True
    else: # Criteo
        finetune_lr = hp["lr_student"] / 10
        grad_noise = 1.2
        embed_noise = 0.1
        finetune_epochs = 15
        use_embed_noise_flag = False

    opt = optim.AdamW(KDkATDP.parameters(), lr=finetune_lr)
    print(f"  Applying DP with grad_noise: {grad_noise}, embed_noise: {embed_noise}, LR: {finetune_lr:.2e}, Epochs: {finetune_epochs}")

    for ep in range(finetune_epochs):
        t0 = time.perf_counter()
        train_loss, train_acc = train_epoch_unified(KDkATDP, tr_loader, opt, CONFIG, dataset_name, teacher=OA, use_at=True, use_dp=True,
                                                      noise_multiplier=grad_noise,
                                                      use_embedding_noise=use_embed_noise_flag,
                                                      embedding_noise_level=embed_noise)
        val_loss, val_acc, _ = evaluate_model_unified(KDkATDP, te_loader, CONFIG, dataset_name)
        _, rob_acc, _ = evaluate_model_unified(KDkATDP, te_loader, CONFIG, dataset_name, robust=True)
        elapsed = time.perf_counter() - t0
        print(f"  Epoch {ep+1:02d}/{finetune_epochs} | Train Loss={train_loss:.4f} | Train ACC={train_acc:6.2f}% | Val ACC={val_acc:6.2f}% | Robust={rob_acc:6.2f}% | Time={elapsed:.1f}s")

    torch.save(KDkATDP.state_dict(), drive_results_dir_tt / f"{dataset_name}_KDkATDP_best.pth")
    SAVED_MODELS_TABTEXT[dataset_name]["KDk+AT+DP"] = f"{dataset_name}_KDkATDP_best.pth"

    del OA, KDk, KDkAT, KDkATDP, opt
    torch.cuda.empty_cache()

print("\n" + "="*70 + "\nTraining completed for all Text/Tabular datasets!")

Starting Text/Tabular VFL Training with Final Optimizations

YAHOO – Phase 1: Training OA (Teacher)


Training YAHOO: 100%|█████████████████████████████████████████| 781/781 [00:06<00:00, 117.55it/s, loss=1.0331]


  Epoch 01/10 | Train Loss=0.9953 | Train ACC= 69.14% | Val Loss=0.8635 | Val ACC= 72.01% | Time=7.6s
    → Best model saved (Val ACC: 72.01%)


Training YAHOO: 100%|██████████████████████████████████████████| 781/781 [00:09<00:00, 83.63it/s, loss=0.8523]


  Epoch 02/10 | Train Loss=0.8209 | Train ACC= 73.36% | Val Loss=0.8518 | Val ACC= 72.27% | Time=10.3s
    → Best model saved (Val ACC: 72.27%)


...

  Epoch 13/15 | Train Loss=0.1685 | Train ACC= 74.58% | Val ACC= 74.95% | Robust= 74.41% | Time=11.6s


Training CRITEO: 100%|██████████████████████████████████████| 1250/1250 [00:09<00:00, 127.29it/s, loss=0.1747]


  Epoch 14/15 | Train Loss=0.1683 | Train ACC= 74.52% | Val ACC= 74.81% | Robust= 74.48% | Time=11.5s


Training CRITEO: 100%|██████████████████████████████████████| 1250/1250 [00:09<00:00, 129.77it/s, loss=0.1610]


  Epoch 15/15 | Train Loss=0.1682 | Train ACC= 74.58% | Val ACC= 74.84% | Robust= 74.47% | Time=11.1s

Training completed for all Text/Tabular datasets!


In [ ]:
# @title
# Cell 11 – Main Text/Tabular Training Loop
import warnings
import time
from pathlib import Path
import torch
import torch.optim as optim

warnings.filterwarnings("ignore", category=UserWarning)

TABTEXT_GROUP = ["YAHOO", "CRITEO"]
SAVED_MODELS_TABTEXT = {}

drive_results_dir_tt = Path(CONFIG["DRIVE_PATH"]) / "VFL_Results" / "TABULAR_TEXT"
drive_results_dir_tt.mkdir(parents=True, exist_ok=True)
print("Starting Text/Tabular VFL Training with Final Optimizations\n" + "="*70)

for dataset_name in TABTEXT_GROUP:
    # Setup
    loaders = TABULAR_TEXT_LOADERS[dataset_name]
    tr_loader, te_loader = loaders["train"], loaders["test"]
    num_classes = loaders["num_classes"]
    epochs = CONFIG["EPOCHS_PER_DATASET"].get(dataset_name, CONFIG["DEFAULT_EPOCHS"])
    hp = CONFIG["DATASET_HP"][dataset_name]
    SAVED_MODELS_TABTEXT[dataset_name] = {}
    patience = 3
    builder_fn = build_vfl_model_tabular_text

    # ---- Phase 1: OA (Teacher) ----
    print(f"\n{dataset_name} – Phase 1: Training OA (Teacher)")
    OA = builder_fn(dataset_name, num_classes)
    opt = optim.AdamW(OA.parameters(), lr=hp["lr_teacher"])
    best_val, no_imp = 0, 0
    for ep in range(epochs):
        t0 = time.perf_counter()
        train_loss, train_acc = train_epoch_unified(OA, tr_loader, opt, CONFIG, dataset_name)
        val_loss, val_acc, _ = evaluate_model_unified(OA, te_loader, CONFIG, dataset_name)
        elapsed = time.perf_counter() - t0
        print(f"  Epoch {ep+1:02d}/{epochs} | Train Loss={train_loss:.4f} | Train ACC={train_acc:6.2f}% | Val Loss={val_loss:.4f} | Val ACC={val_acc:6.2f}% | Time={elapsed:.1f}s")

        if val_acc > best_val:
            best_val, no_imp = val_acc, 0
            torch.save(OA.state_dict(), drive_results_dir_tt / f"{dataset_name}_OA_best.pth")
            print(f"    → Best model saved (Val ACC: {val_acc:.2f}%)")
        elif ep > 1:
            no_imp += 1
        if no_imp >= patience:
            print("  Early stopping triggered.")
            break
    OA.load_state_dict(torch.load(drive_results_dir_tt / f"{dataset_name}_OA_best.pth", map_location=CONFIG["DEVICE"]))
    SAVED_MODELS_TABTEXT[dataset_name]["OA"] = f"{dataset_name}_OA_best.pth"

    # ---- Phase 2: KDk ----
    print(f"\n{dataset_name} – Phase 2: Training KDk")
    KDk = builder_fn(dataset_name, num_classes)
    opt = optim.AdamW(KDk.parameters(), lr=hp["lr_student"])
    best_val, no_imp = 0, 0
    for ep in range(epochs):
        t0 = time.perf_counter()
        train_loss, train_acc = train_epoch_unified(KDk, tr_loader, opt, CONFIG, dataset_name, teacher=OA)
        val_loss, val_acc, _ = evaluate_model_unified(KDk, te_loader, CONFIG, dataset_name)
        elapsed = time.perf_counter() - t0
        print(f"  Epoch {ep+1:02d}/{epochs} | Train Loss={train_loss:.4f} | Train ACC={train_acc:6.2f}% | Val Loss={val_loss:.4f} | Val ACC={val_acc:6.2f}% | Time={elapsed:.1f}s")

        if val_acc > best_val:
            best_val, no_imp = val_acc, 0
            torch.save(KDk.state_dict(), drive_results_dir_tt / f"{dataset_name}_KDk_best.pth")
            print(f"    → Best model saved (Val ACC: {val_acc:.2f}%)")
        elif ep > 1:
            no_imp += 1
        if no_imp >= patience:
            print("  Early stopping triggered.")
            break
    KDk.load_state_dict(torch.load(drive_results_dir_tt / f"{dataset_name}_KDk_best.pth", map_location=CONFIG["DEVICE"]))
    SAVED_MODELS_TABTEXT[dataset_name]["KDk"] = f"{dataset_name}_KDk_best.pth"

    # ---- Phase 3: KDk+AT ----
    print(f"\n{dataset_name} – Phase 3: Training KDk+AT")
    KDkAT = builder_fn(dataset_name, num_classes)
    opt = optim.AdamW(KDkAT.parameters(), lr=hp["lr_student"])
    best_rob, no_imp = 0, 0
    for ep in range(epochs):
        t0 = time.perf_counter()
        train_loss, train_acc = train_epoch_unified(KDkAT, tr_loader, opt, CONFIG, dataset_name, teacher=OA, use_at=True)
        val_loss, val_acc, _ = evaluate_model_unified(KDkAT, te_loader, CONFIG, dataset_name)
        _, rob_acc, _ = evaluate_model_unified(KDkAT, te_loader, CONFIG, dataset_name, robust=True)
        elapsed = time.perf_counter() - t0
        print(f"  Epoch {ep+1:02d}/{epochs} | Train Loss={train_loss:.4f} | Train ACC={train_acc:6.2f}% | Val ACC={val_acc:6.2f}% | Robust={rob_acc:6.2f}% | Time={elapsed:.1f}s")

        if rob_acc > best_rob:
            best_rob, no_imp = rob_acc, 0
            torch.save(KDkAT.state_dict(), drive_results_dir_tt / f"{dataset_name}_KDkAT_best.pth")
            print(f"    → Best model saved (Robust ACC: {rob_acc:.2f}%)")
        elif ep > 1:
            no_imp += 1
        if no_imp >= patience:
            print("  Early stopping triggered.")
            break
    KDkAT.load_state_dict(torch.load(drive_results_dir_tt / f"{dataset_name}_KDkAT_best.pth", map_location=CONFIG["DEVICE"]))
    SAVED_MODELS_TABTEXT[dataset_name]["KDk+AT"] = f"{dataset_name}_KDkAT_best.pth"

    # ---- Phase 4: KDk+AT+DP (with AGGRESSIVE HYPERPARAMETERS) ----
    print(f"\n{dataset_name} – Phase 4: Training KDk+AT+DP")
    KDkATDP = builder_fn(dataset_name, num_classes)
    KDkATDP.load_state_dict(torch.load(drive_results_dir_tt / f"{dataset_name}_KDkAT_best.pth", map_location=CONFIG["DEVICE"]))

    if dataset_name == "YAHOO":
        finetune_lr = hp["lr_student"] / 25
        grad_noise = 4.0
        embed_noise = 0.3
        finetune_epochs = 20
        use_embed_noise_flag = True
    else: # Criteo
        finetune_lr = hp["lr_student"] / 10
        grad_noise = 1.2
        embed_noise = 0.1
        finetune_epochs = 15
        use_embed_noise_flag = False

    opt = optim.AdamW(KDkATDP.parameters(), lr=finetune_lr)
    print(f"  Applying DP with grad_noise: {grad_noise}, embed_noise: {embed_noise}, LR: {finetune_lr:.2e}, Epochs: {finetune_epochs}")

    for ep in range(finetune_epochs):
        t0 = time.perf_counter()
        train_loss, train_acc = train_epoch_unified(KDkATDP, tr_loader, opt, CONFIG, dataset_name, teacher=OA, use_at=True, use_dp=True,
                                                      noise_multiplier=grad_noise,
                                                      use_embedding_noise=use_embed_noise_flag,
                                                      embedding_noise_level=embed_noise)
        val_loss, val_acc, _ = evaluate_model_unified(KDkATDP, te_loader, CONFIG, dataset_name)
        _, rob_acc, _ = evaluate_model_unified(KDkATDP, te_loader, CONFIG, dataset_name, robust=True)
        elapsed = time.perf_counter() - t0
        print(f"  Epoch {ep+1:02d}/{finetune_epochs} | Train Loss={train_loss:.4f} | Train ACC={train_acc:6.2f}% | Val ACC={val_acc:6.2f}% | Robust={rob_acc:6.2f}% | Time={elapsed:.1f}s")

    torch.save(KDkATDP.state_dict(), drive_results_dir_tt / f"{dataset_name}_KDkATDP_best.pth")
    SAVED_MODELS_TABTEXT[dataset_name]["KDk+AT+DP"] = f"{dataset_name}_KDkATDP_best.pth"

    del OA, KDk, KDkAT, KDkATDP, opt
    torch.cuda.empty_cache()

print("\n" + "="*70 + "\nTraining completed for all Text/Tabular datasets!")

Starting Text/Tabular VFL Training with Final Optimizations

YAHOO – Phase 1: Training OA (Teacher)


Training YAHOO: 100%|█████████████████████████████████████████| 781/781 [00:04<00:00, 178.23it/s, loss=1.1236]


  Epoch 01/10 | Train Loss=0.9977 | Train ACC= 69.01% | Val Loss=0.8636 | Val ACC= 72.15% | Time=5.1s
    → Best model saved (Val ACC: 72.15%)


Training YAHOO: 100%|█████████████████████████████████████████| 781/781 [00:04<00:00, 158.61it/s, loss=0.9291]


  Epoch 02/10 | Train Loss=0.8240 | Train ACC= 73.30% | Val Loss=0.8450 | Val ACC= 72.67% | Time=5.5s
    → Best model saved (Val ACC: 72.67%)


...

  Epoch 13/15 | Train Loss=0.1711 | Train ACC= 74.39% | Val ACC= 75.32% | Robust= 74.30% | Time=11.3s


Training CRITEO: 100%|██████████████████████████████████████| 1250/1250 [00:10<00:00, 119.64it/s, loss=0.2065]


  Epoch 14/15 | Train Loss=0.1710 | Train ACC= 74.42% | Val ACC= 75.16% | Robust= 74.34% | Time=11.7s


Training CRITEO: 100%|██████████████████████████████████████| 1250/1250 [00:10<00:00, 119.87it/s, loss=0.1811]


  Epoch 15/15 | Train Loss=0.1709 | Train ACC= 74.48% | Val ACC= 75.28% | Robust= 74.38% | Time=11.7s

Training completed for all Text/Tabular datasets!


In [ ]:
# Cell 7 - Focused Evaluation for Text/Tabular
from sklearn.metrics import f1_score, roc_auc_score
import pandas as pd
from IPython.display import display
import numpy as np
from pathlib import Path
import torch

all_evaluation_results = []
print("--- FOCUSED EVALUATION INITIATED for Text/Tabular ---")

SAVED_MODELS_TABTEXT = {
    "YAHOO": { "OA": "YAHOO_OA_best.pth", "KDk": "YAHOO_KDk_best.pth", "KDk+AT": "YAHOO_KDkAT_best.pth", "KDk+AT+DP": "YAHOO_KDkATDP_best.pth" },
    "CRITEO": { "OA": "CRITEO_OA_best.pth", "KDk": "CRITEO_KDk_best.pth", "KDk+AT": "CRITEO_KDkAT_best.pth", "KDk+AT+DP": "CRITEO_KDkATDP_best.pth" }
}

model_group = { "type": "TABULAR_TEXT", "models": SAVED_MODELS_TABTEXT, "loaders": TABULAR_TEXT_LOADERS, "builder": build_vfl_model_tabular_text }
model_type_name = model_group["type"]
drive_dir = Path(CONFIG["DRIVE_PATH"]) / "VFL_Results" / model_type_name

for dataset_name, models in model_group["models"].items():
    loaders = model_group["loaders"][dataset_name]
    tr_loader, te_loader = loaders["train"], loaders["test"]
    num_classes = loaders["num_classes"]
    builder_fn = model_group["builder"]

    for model_type, model_path in models.items():
        print(f"Evaluating {dataset_name} - {model_type}...")
        model = builder_fn(dataset_name, num_classes)
        model.load_state_dict(torch.load(drive_dir / model_path, map_location=CONFIG["DEVICE"]))

        _, clean_acc_top1, _ = evaluate_model_unified(model, te_loader, CONFIG, dataset_name)
        _, robust_acc_top1, _ = evaluate_model_unified(model, te_loader, CONFIG, dataset_name, robust=True)

        y_true_list, y_probs_list = [], []
        with torch.no_grad():
            for (xa, xp), y in te_loader:
                xa, xp = xa.to(CONFIG["DEVICE"]), xp.to(CONFIG["DEVICE"])
                logits = model(xa, xp)
                y_true_list.extend(y.numpy())
                y_probs_list.extend(torch.softmax(logits, dim=1).cpu().numpy())
        y_true, y_probs = np.array(y_true_list), np.array(y_probs_list)
        y_pred = np.argmax(y_probs, axis=1)

        f1 = f1_score(y_true, y_pred, average='binary' if num_classes == 2 else 'macro') * 100
        auc = roc_auc_score(y_true, y_probs[:, 1]) * 100 if num_classes == 2 else roc_auc_score(y_true, y_probs, multi_class='ovr') * 100

        attack_results = {}
        for attack in ["passive", "direct", "active", "perturbed"]:
            a1, _ = run_privacy_attack_unified( model, tr_loader, te_loader, CONFIG, num_classes, attack_type=attack, aux_batches_limit=4, attacker_epochs=2)
            attack_results[attack] = {"top1": a1}

        pli = (1 - attack_results["passive"]["top1"]/100.0) * (robust_acc_top1 / max(clean_acc_top1,1)) * 100

        all_evaluation_results.append({
            "Dataset": dataset_name, "Model": model_type,
            "Clean ACC (Top-1 %)": round(clean_acc_top1, 2),
            "Robust ACC (Top-1 %)": round(robust_acc_top1, 2),
            "F1 Score (%)": round(f1, 2),
            "AUC (%)": round(auc, 2),
            "ASR_passive (Top-1 %)": round(attack_results["passive"]["top1"], 2),
            "ASR_direct (Top-1 %)": round(attack_results["direct"]["top1"], 2),
            "ASR_active (Top-1 %)": round(attack_results["active"]["top1"], 2),
            "ASR_perturbed (Top-1 %)": round(attack_results["perturbed"]["top1"], 2),
            "Privacy Leakage Index": round(pli, 2)
        })

final_df = pd.DataFrame(all_evaluation_results)
final_path = drive_dir / "evaluation_results_TextTabular_final.csv"
final_df.to_csv(final_path, index=False)
print("\n--- EVALUATION COMPLETE ---"); display(final_df); print(f"\nText/Tabular results saved to: {final_path}")

--- FOCUSED EVALUATION INITIATED for Text/Tabular ---
Evaluating YAHOO - OA...
Evaluating YAHOO - KDk...
Evaluating YAHOO - KDk+AT...
Evaluating YAHOO - KDk+AT+DP...
Evaluating CRITEO - OA...
Evaluating CRITEO - KDk...
Evaluating CRITEO - KDk+AT...
Evaluating CRITEO - KDk+AT+DP...

--- EVALUATION COMPLETE ---


,Dataset,Model,Clean ACC (Top-1 %),Robust ACC (Top-1 %),F1 Score (%),AUC (%),ASR_passive (Top-1 %),ASR_direct (Top-1 %),ASR_active (Top-1 %),ASR_perturbed (Top-1 %),Privacy Leakage Index
0,YAHOO,OA,72.67,67.49,72.05,95.13,15.15,19.73,20.13,13.08,78.80
1,YAHOO,KDk,72.84,66.02,72.28,95.20,18.20,23.88,10.98,12.45,74.14
2,YAHOO,KDk+AT,72.75,70.43,72.17,95.19,14.41,21.61,24.98,9.71,82.86
3,YAHOO,KDk+AT+DP,72.78,70.41,72.18,95.20,14.71,18.35,10.14,10.30,82.51
4,CRITEO,OA,75.74,71.58,29.84,70.87,66.75,73.84,62.09,41.97,31.43
5,CRITEO,KDk,75.67,72.69,23.01,70.44,56.25,73.14,70.25,74.30,42.03
6,CRITEO,KDk+AT,74.83,74.52,9.49,69.57,74.47,74.16,74.44,74.47,25.43
7,CRITEO,KDk+AT+DP,75.28,74.38,17.65,69.60,74.26,74.47,71.88,74.44,25.44



Text/Tabular results saved to: /content/VFL_Results/TABULAR_TEXT/evaluation_results_TextTabular_final.csv
